In [1]:
# 서울버스정류장
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)

# 2021.09.24일 수집 완료
query = ("select seoulsttsName from public_seoulbusstts ps where 10026 <= seq and seq <= 10921")

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
now = datetime.now()
nowDate = now.strftime('%Y%m%d')

cnt = 1
for i in location_list:
    seoulsttsName = i
    
    characters = "(',')"

    join_data = ''.join( x for x in seoulsttsName if x not in characters)
    print("조회날짜 : {} | 순번 : {} | start {} public_seoulbusstts".format(nowDate, cnt, join_data))
    url = 'http://ws.bus.go.kr/api/rest/stationinfo/getStationByName'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('stSrch') : join_data
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('itemList')
    resultCode = xmlobj.findAll('headerCd')

    if resultCode[0].text == '7':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(seoulsttsName))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        arsId = item.findAll('arsId')
        stId = item.findAll('stId')
        stNm = item.findAll('stNm')
        lat = item.findAll('tmY')
        lng = item.findAll('tmX')
        
        try:
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
        except IndexError:
            print(item)
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210924 | 순번 : 1 | start 서울교대사거리 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 2 | start 서울교대사거리 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 3 | start 서울교대사거리 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 4 | start 지하철3호선교대역 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 5 | start KCC사옥 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 6 | start 서초푸르지오써밋 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 7 | start 서초푸르지오써밋.롯데캐슬클래식 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 8 | start 진흥아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 9 | start 진흥아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 10 | start 강남역.강남역사거리 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 11 | start 서초동진흥아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 12 | start 서초동유원아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 13 | start 지하철2호선교대역4출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 14 | start 법조단지 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 15 | start 신논현역.구교보타워사거리 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 16 | start 강남역 public_seoulbusstts
조회

조회날짜 : 20210924 | 순번 : 131 | start 구룡사 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 132 | start 구룡사입구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 133 | start 동원산업앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 134 | start 언남고포이성당앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 135 | start 양재119안전센터앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 136 | start 소방서앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 137 | start 꽃마을 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 138 | start 꽃마을 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 139 | start 개나리골입구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 140 | start 개나리골입구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 141 | start 새원마을 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 142 | start 새원마을 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 143 | start 본마을 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 144 | start 본마을 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 145 | start 청계산.원터골 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 146 | start 청계산.원터골 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 

조회날짜 : 20210924 | 순번 : 263 | start 사당역13번출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 264 | start 서초구립반포도서관.반포리체 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 265 | start 서초문화예술정보학교후문 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 266 | start 장자터빌딩 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 267 | start 동부센트레빌1 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 268 | start 필리핀대사관 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 269 | start 무지개아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 270 | start 삼성래미안 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 271 | start kt연구소 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 272 | start 상문고등학교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 273 | start 반포리체 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 274 | start 백산빌딩 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 275 | start 방배브라운가아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 276 | start 방배열린문화센터.가야치과병원 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 277 | start 방배중학교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 278 | start 반포래미안 public_s

조회날짜 : 20210924 | 순번 : 395 | start 네이처힐.LH서초아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 396 | start 서초호반써밋 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 397 | start 한신2차후문 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 398 | start 서초아트자이아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 399 | start 방배반딧불센터 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 400 | start 신사동.더리버사이드호텔 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 401 | start 교대역 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 402 | start 서초고후문 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 403 | start 우성아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 404 | start 내방역1번출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 405 | start 교보타워앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 406 | start 잠원역 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 407 | start 동원산업 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 408 | start 서울시인재개발원 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 409 | start 한신2차정문 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 410 | start 미켈란아파트 public_seoulbusstts


조회날짜 : 20210924 | 순번 : 528 | start 서울고사거리 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 529 | start 사당역12번출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 530 | start 삼성래미안.이수역방향 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 531 | start 더미켈란아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 532 | start 서울성모병원.서울지방조달청 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 533 | start 아크로비스타 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 534 | start 서초성당 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 535 | start 강남역나라빌딩앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 536 | start 양재역신한은행앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 537 | start 신사중학교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 538 | start 신사중학교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 539 | start 광림교회.현대고등학교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 540 | start 광림교회.현대고등학교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 541 | start 압구정역4번출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 542 | start 국민은행 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 543 | start 신사동고개 pub

조회날짜 : 20210924 | 순번 : 657 | start 한티역3번출구.서울강남고용노동지청 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 658 | start 삼성아파트입구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 659 | start 농협대치지점 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 660 | start 대치SK.VIEW아파트.서울교회 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 661 | start 대치역1번출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 662 | start 래미안.대치.팰리스 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 663 | start 도곡역3번출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 664 | start 대치동우성아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 665 | start 대치역8번출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 666 | start 대치역7번출구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 667 | start 그랑프리백화점 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 668 | start 은마아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 669 | start 은마파출소 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 670 | start 은마아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 671 | start 대치동은마아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 672 | start 은

조회날짜 : 20210924 | 순번 : 785 | start 경기여고 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 786 | start 개포동역.개포시장 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 787 | start 개포주공5단지경기여고 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 788 | start 개포동역.개포시장 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 789 | start 개포시장.개포5단지상가 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 790 | start 개포시장.개포5단지상가 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 791 | start 양전초등학교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 792 | start 개포6단지앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 793 | start 공무원아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 794 | start 개원중학교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 795 | start 개포주공7단지앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 796 | start 연금매점 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 797 | start 현대아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 798 | start 개포래미안포레스트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 799 | start 능인선원앞 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 800 | start 구룡산입구 public_s

조회날짜 : 20210924 | 순번 : 916 | start 개포4단지7단지 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 917 | start 총회회관.휘문고입구 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 918 | start 예림당아트홀.신한은행전산센터 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 919 | start 새마을운동중앙회 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 920 | start 총지사 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 921 | start 신한은행데이터센터 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 922 | start 포스코사거리 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 923 | start 우성3차아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 924 | start 우성6차아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 925 | start 포이초교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 926 | start 국악중고교 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 927 | start 풍양빌딩 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 928 | start 코원에너지서비스 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 929 | start 코원에너지서비스 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 930 | start 현대1차우성아파트 public_seoulbusstts
조회날짜 : 20210924 | 순번 : 931 | start 포스코사거리 public_